<a href="https://colab.research.google.com/github/codesongs/codestates_TP2/blob/main/%5BO%5D_230830_AIB_recsys_b_k.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# surprise 라이브러리 설치
!pip install scikit-surprise

In [ ]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader, Dataset, SVD, accuracy

import warnings; warnings.simplefilter('ignore')

## 1. 단순추천시스템

In [ ]:
md = pd.read_csv('b_k_mov_rat.csv')
md = md[['MovieID', 'title', 'year', 'id', 'genres', 'popularity',
       'overview', 'tagline', 'b_vote_average', 'b_vote_count', 'cast', 'crew',
       'keywords']]
md.head()

,MovieID,title,year,id,genres,popularity,overview,tagline,b_vote_average,b_vote_count,cast,crew,keywords
0,1,Toy Story,1995,862.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",21.946943,"Led by Woody, Andy's toys live happily in his ...",0,4.146846,2077.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,2,Jumanji,1995,8844.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",17.015539,When siblings Judy and Peter discover an encha...,Roll the dice and unleash the excitement!,3.201141,701.0,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,3,Grumpier Old Men,1995,15602.0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",11.712900,A family wedding reignites the ancient feud be...,Still Yelling. Still Fighting. Still Ready for...,3.016736,478.0,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,4,Waiting to Exhale,1995,31357.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",3.859495,"Cheated on, mistreated and stepped on, the wom...",Friends are the people who let you be yourself...,2.729412,170.0,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...","[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,5,Father of the Bride Part II,1995,11862.0,"[{'id': 35, 'name': 'Comedy'}]",8.387519,Just when George Banks has recovered from his ...,Just When His World Is Back To Normal... He's ...,3.006757,296.0,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...","[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [ ]:
md.rename(columns={'b_vote_average': 'vote_average'}, inplace=True)
md.rename(columns={'b_vote_count': 'vote_count'}, inplace=True)

In [ ]:
md.columns

Index(['MovieID', 'title', 'year', 'id', 'genres', 'popularity', 'overview',
       'tagline', 'vote_average', 'vote_count', 'cast', 'crew', 'keywords'],
      dtype='object')

In [ ]:
print('////////////////////// Before ////////////////////// \n', md['genres'].head(), '\n////////////////////// Before //////////////////////')

# dictionary안에 담겨있는 Genre 정보를 List 형태로 세팅
# 1. md['genres'].fillna('[]') : genres 컬럼에 null 값을 '[](빈 리스트 값)'으로 채워넣음
# 2. apply(literal_eval) : literal_eval를 사용하여 String으로 되어있는 값을 List & Dictionary로 사용할 수 있게 변환
# 3. apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else []) : x가 list인 경우 안에 들어있는 dictionary 중 name에 해당하는 값을 list에 담음
md['genres'] = md['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

print('////////////////////// After ////////////////////// \n', md['genres'].head(), '\n////////////////////// After //////////////////////')

////////////////////// Before ////////////////////// 
 0    [{'id': 16, 'name': 'Animation'}, {'id': 35, '...
1    [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...
2    [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...
3    [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...
4                       [{'id': 35, 'name': 'Comedy'}]
Name: genres, dtype: object 
////////////////////// Before //////////////////////
////////////////////// After ////////////////////// 
 0     [Animation, Comedy, Family]
1    [Adventure, Fantasy, Family]
2               [Romance, Comedy]
3        [Comedy, Drama, Romance]
4                        [Comedy]
Name: genres, dtype: object 
////////////////////// After //////////////////////


In [ ]:
print('vote ::: \n', md[['vote_count', 'vote_average']].head())
vote_counts = md[md['vote_count'].notnull()]['vote_count'].astype('int')
vote_averages = md[md['vote_average'].notnull()]['vote_average'].astype('int')
C = vote_averages.mean()
C

vote ::: 
    vote_count  vote_average
0      2077.0      4.146846
1       701.0      3.201141
2       478.0      3.016736
3       170.0      2.729412
4       296.0      3.006757


2.762277388019428

In [ ]:
# 총 영화 중 상위 5%
print(vote_counts.sort_values(ascending=False)[2273:2274])

# quantile는 데이터를 크기대로 정렬하였을 때 분위수를 구하는 함수. quantile(0.95)는 상위 5%에 해당하는 값을 찾는 것
m = vote_counts.quantile(0.95)
m

2350    69
Name: vote_count, dtype: int64


1051.5

In [ ]:
qualified = md

In [ ]:
def weighted_rating(x):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)

In [ ]:
qualified['wr'] = qualified.apply(weighted_rating, axis=1)

In [ ]:
# Weighted Rating 상위 250개의 영화
qualified = qualified.sort_values('wr', ascending=False).head(250)

**Top Movies**

In [ ]:
qualified.head(15)

,MovieID,title,year,id,genres,popularity,overview,tagline,vote_average,vote_count,cast,crew,keywords,wr
257,260,Star Wars: Episode IV - A New Hope,1977,NaN,[],NaN,NaN,NaN,4.453694,2991.0,NaN,NaN,NaN,4.013738
315,318,"Shawshank Redemption, The",1994,NaN,[],NaN,NaN,NaN,4.554558,2227.0,NaN,NaN,NaN,3.979727
1180,1198,Raiders of the Lost Ark,1981,85.0,"[Adventure, Action]",19.901576,When Dr. Indiana Jones – the tweed-suited prof...,Indiana Jones - the new hero from the creators...,4.477725,2514.0,"[{'cast_id': 2, 'character': 'Indy', 'credit_i...","[{'credit_id': '52fe4215c3a36847f8002a5b', 'de...","[{'id': 83, 'name': 'saving the world'}, {'id'...",3.971823
523,527,Schindler's List,1993,424.0,"[Drama, History, War]",41.725123,The true story of how businessman Oskar Schind...,"Whoever saves one life, saves the world entire.",4.510417,2304.0,"[{'cast_id': 14, 'character': 'Oskar Schindler...","[{'credit_id': '52fe4241c3a36847f801024d', 'de...","[{'id': 1382, 'name': 'factory'}, {'id': 1631,...",3.962609
847,858,"Godfather, The",1972,NaN,[],NaN,NaN,NaN,4.524966,2223.0,NaN,NaN,NaN,3.958936
2789,2858,American Beauty,1999,14.0,[Drama],20.726578,"Lester Burnham, a depressed suburban father in...",Look closer.,4.317386,3428.0,"[{'cast_id': 6, 'character': 'Lester Burnham',...","[{'credit_id': '52fe420ec3a36847f80007c5', 'de...","[{'id': 255, 'name': 'male nudity'}, {'id': 29...",3.952346
2693,2762,"Sixth Sense, The",1999,NaN,[],NaN,NaN,NaN,4.406263,2459.0,NaN,NaN,NaN,3.913840
1178,1196,Star Wars: Episode V - The Empire Strikes Back,1980,NaN,[],NaN,NaN,NaN,4.292977,2990.0,NaN,NaN,NaN,3.894726
589,593,"Silence of the Lambs, The",1991,NaN,[],NaN,NaN,NaN,4.351823,2578.0,NaN,NaN,NaN,3.891317
1959,2028,Saving Private Ryan,1998,857.0,"[Drama, History, War]",21.758054,"As U.S. troops storm the beaches of Normandy, ...",The mission is a man.,4.337354,2653.0,"[{'cast_id': 10, 'character': 'Captain John H....","[{'credit_id': '52fe4283c3a36847f8024abb', 'de...","[{'id': 1327, 'name': 'war crimes'}, {'id': 14...",3.890278


In [ ]:
# stack() : stack이 (위에서 아래로 길게, 높게) 쌓는 것이면, unstack은 쌓은 것을 옆으로 늘어놓는것(왼쪽에서 오른쪽으로 넓게) 라고 연상이 될 것
# reset_index() : 기존의 행 인덱스를 제거하고 인덱스를 데이터 열로 추가
s = md.apply(lambda x: pd.Series(x['genres']), axis=1).stack().reset_index(level=1, drop=True)
s.name = 'genre'
print(s.head(10))

gen_md = md.drop('genres', axis=1).join(s)
print(gen_md.head(10))

0    Animation
0       Comedy
0       Family
1    Adventure
1      Fantasy
1       Family
2      Romance
2       Comedy
3       Comedy
3        Drama
Name: genre, dtype: object
   MovieID              title  year       id  popularity  \
0        1          Toy Story  1995    862.0   21.946943   
0        1          Toy Story  1995    862.0   21.946943   
0        1          Toy Story  1995    862.0   21.946943   
1        2            Jumanji  1995   8844.0   17.015539   
1        2            Jumanji  1995   8844.0   17.015539   
1        2            Jumanji  1995   8844.0   17.015539   
2        3   Grumpier Old Men  1995  15602.0   11.712900   
2        3   Grumpier Old Men  1995  15602.0   11.712900   
3        4  Waiting to Exhale  1995  31357.0    3.859495   
3        4  Waiting to Exhale  1995  31357.0    3.859495   

                                            overview  \
0  Led by Woody, Andy's toys live happily in his ...   
0  Led by Woody, Andy's toys live happily in his .

In [ ]:
def build_chart(genre, percentile=0.85):
    df = gen_md[gen_md['genre'] == genre]
    vote_counts = df[df['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = df[df['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(percentile)

    qualified = df[(df['vote_count'] >= m) & (df['vote_count'].notnull()) & (df['vote_average'].notnull())][['title','year','vote_count','vote_average','popularity']]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')

    qualified['wr'] = qualified.apply(lambda x: (x['vote_count']/(x['vote_count']+m) * x['vote_average']) + (m/(m+x['vote_count']) * C), axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(250)

    return qualified

**Top Romance Movies**

In [ ]:
build_chart('Romance').head(15)

,title,year,vote_count,vote_average,popularity,wr
2327,Shakespeare in Love,1998,2369,4,13.813155,3.795626
352,Forrest Gump,1994,2194,4,48.307194,3.782350
900,Casablanca,1942,1669,4,13.916101,3.729669
1287,When Harry Met Sally...,1989,1568,4,8.343220,3.716466
1211,Annie Hall,1977,1334,4,18.146043,3.680290
2179,Say Anything...,1989,914,4,6.953646,3.585326
891,Vertigo,1958,905,4,18.208220,3.582670
16,Sense and Sensibility,1995,835,4,10.673167,3.560788
898,Some Like It Hot,1959,832,4,11.845107,3.559798
887,Singin' in the Rain,1952,751,4,11.064858,3.531296


평가지표 (Ndcg, Recall)

In [ ]:
import numpy as np

def calculate_ndcg(predictions, k=10):
    actual_scores = np.array(predictions['vote_average'])
    predicted_scores = np.array(predictions['wr'])

    sorted_actual_scores = np.sort(actual_scores)[::-1]
    sorted_predicted_scores = predicted_scores[np.argsort(actual_scores)[::-1]]

    dcg = np.sum((2**sorted_actual_scores - 1) / np.log2(np.arange(2, k + 2)))
    idcg = np.sum((2**sorted_actual_scores - 1) / np.log2(np.arange(2, k + 2)))

    ndcg = dcg / idcg if idcg > 0 else 0
    return ndcg

def calculate_recall(predictions, threshold=7.0):
    true_positives = len(predictions[predictions['vote_average'] >= threshold])
    total_positives = len(predictions)

    recall = true_positives / total_positives if total_positives > 0 else 0
    return recall

# 예시: 상위 10개 영화에 대한 NDCG 및 Recall 계산
top_10_predictions = qualified.head(10)
ndcg_10 = calculate_ndcg(top_10_predictions, k=10)
recall_10 = calculate_recall(top_10_predictions, threshold=7.0)

print("NDCG@10:", ndcg_10)
print("Recall@10:", recall_10)

NDCG@10: 1.0
Recall@10: 0.0


##2. 콘텐츠 기반 추천(Content Based Recommender)

In [ ]:
import pandas as pd

In [ ]:
df_cbrmd = pd.read_csv('b_k_mov_rat.csv')
df_cbrmd.head()

,MovieID,title,year,Genres,id,genres,popularity,overview,tagline,b_vote_average,b_vote_count,cast,crew,keywords
0,1,Toy Story,1995,Animation|Children's|Comedy,862.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",21.946943,"Led by Woody, Andy's toys live happily in his ...",0,4.146846,2077.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,2,Jumanji,1995,Adventure|Children's|Fantasy,8844.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",17.015539,When siblings Judy and Peter discover an encha...,Roll the dice and unleash the excitement!,3.201141,701.0,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,3,Grumpier Old Men,1995,Comedy|Romance,15602.0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",11.712900,A family wedding reignites the ancient feud be...,Still Yelling. Still Fighting. Still Ready for...,3.016736,478.0,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,4,Waiting to Exhale,1995,Comedy|Drama,31357.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",3.859495,"Cheated on, mistreated and stepped on, the wom...",Friends are the people who let you be yourself...,2.729412,170.0,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...","[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,5,Father of the Bride Part II,1995,Comedy,11862.0,"[{'id': 35, 'name': 'Comedy'}]",8.387519,Just when George Banks has recovered from his ...,Just When His World Is Back To Normal... He's ...,3.006757,296.0,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...","[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [ ]:
# 전처리
# overview의 결측치가 있는 항목은 모두 제거
df_cbrmd = df_cbrmd[df_cbrmd['overview'].notnull()].reset_index(drop=True)
df_cbrmd.shape

(2670, 14)

In [ ]:
# 불용어 : 유의미하지 않은 단어 토큰을 제거
# https://wikidocs.net/22530
# tfidf = TfidfVectorizer(stop_words='english')
tfidf = TfidfVectorizer(stop_words='english')

# overview에 대해서 tf-idf 수행
tfidf_matrix = tfidf.fit_transform(df_cbrmd['overview'])
print(tfidf_matrix.shape)

(2670, 15952)


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)

# 만일 여기서 메모리 에러가 발생하신 분은 TF-IDF의 파라미터를 수정해줘서 다시 돌리면 됩니다.
# tfidf = TfidfVectorizer(stop_words='english', max_features=10000)

# 그래도, 안되는 경우에는 문서의 수를 조금 줄여서 실행해보시길 바랍니다.
# data = data.loc[0:10000].reset_index(drop=True)

In [ ]:
np.round(cosine_matrix, 4)

array([[1.    , 0.0185, 0.    , ..., 0.    , 0.    , 0.    ],
       [0.0185, 1.    , 0.0458, ..., 0.    , 0.    , 0.    ],
       [0.    , 0.0458, 1.    , ..., 0.    , 0.    , 0.0119],
       ...,
       [0.    , 0.    , 0.    , ..., 1.    , 0.    , 0.0202],
       [0.    , 0.    , 0.    , ..., 0.    , 1.    , 0.    ],
       [0.    , 0.    , 0.0119, ..., 0.0202, 0.    , 1.    ]])

In [ ]:
# movie title와 id를 매핑할 dictionary를 생성해줍니다.
movie2id = {}
for i, c in enumerate(df_cbrmd['title']): movie2id[i] = c

# id와 movie title를 매핑할 dictionary를 생성해줍니다.
id2movie = {}
for i, c in movie2id.items(): id2movie[c] = i

In [ ]:
movie2id

{0: 'Toy Story',
 1: 'Jumanji',
 2: 'Grumpier Old Men',
 3: 'Waiting to Exhale',
 4: 'Father of the Bride Part II',
 5: 'Heat',
 6: 'Sabrina',
 7: 'Tom and Huck',
 8: 'Sudden Death',
 9: 'GoldenEye',
 10: 'Dracula: Dead and Loving It',
 11: 'Balto',
 12: 'Nixon',
 13: 'Cutthroat Island',
 14: 'Casino',
 15: 'Sense and Sensibility',
 16: 'Four Rooms',
 17: 'Ace Ventura: When Nature Calls',
 18: 'Money Train',
 19: 'Get Shorty',
 20: 'Copycat',
 21: 'Assassins',
 22: 'Powder',
 23: 'Leaving Las Vegas',
 24: 'Othello',
 25: 'Now and Then',
 26: 'Persuasion',
 27: 'Shanghai Triad',
 28: 'Dangerous Minds',
 29: 'Twelve Monkeys',
 30: 'Wings of Courage',
 31: 'Babe',
 32: 'Carrington',
 33: 'Dead Man Walking',
 34: 'Across the Sea of Time',
 35: 'It Takes Two',
 36: 'Clueless',
 37: 'Cry, the Beloved Country',
 38: 'Richard III',
 39: 'Dead Presidents',
 40: 'Restoration',
 41: 'Mortal Kombat',
 42: 'To Die For',
 43: 'Seven',
 44: 'Pocahontas',
 45: 'When Night Is Falling',
 46: 'Guardian A

In [ ]:
id2movie

{'Toy Story': 0,
 'Jumanji': 1,
 'Grumpier Old Men': 2,
 'Waiting to Exhale': 3,
 'Father of the Bride Part II': 4,
 'Heat': 5,
 'Sabrina': 644,
 'Tom and Huck': 7,
 'Sudden Death': 8,
 'GoldenEye': 9,
 'Dracula: Dead and Loving It': 10,
 'Balto': 11,
 'Nixon': 12,
 'Cutthroat Island': 13,
 'Casino': 14,
 'Sense and Sensibility': 15,
 'Four Rooms': 16,
 'Ace Ventura: When Nature Calls': 17,
 'Money Train': 18,
 'Get Shorty': 19,
 'Copycat': 20,
 'Assassins': 21,
 'Powder': 22,
 'Leaving Las Vegas': 23,
 'Othello': 1920,
 'Now and Then': 25,
 'Persuasion': 26,
 'Shanghai Triad': 27,
 'Dangerous Minds': 28,
 'Twelve Monkeys': 29,
 'Wings of Courage': 30,
 'Babe': 31,
 'Carrington': 32,
 'Dead Man Walking': 33,
 'Across the Sea of Time': 34,
 'It Takes Two': 35,
 'Clueless': 36,
 'Cry, the Beloved Country': 37,
 'Richard III': 38,
 'Dead Presidents': 39,
 'Restoration': 40,
 'Mortal Kombat': 41,
 'To Die For': 42,
 'Seven': 43,
 'Pocahontas': 44,
 'When Night Is Falling': 45,
 'Guardian A

In [ ]:
# Toy Story의 id 추출
idx = id2movie['Toy Story'] # Toy Story : 0번 인덱스
sim_scores = [(i, c) for i, c in enumerate(cosine_matrix[idx]) if i != idx] # 자기 자신을 제외한 영화들의 유사도 및 인덱스를 추출
sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse=True) # 유사도가 높은 순서대로 정렬
sim_scores[0:10] # 상위 10개의 인덱스와 유사도를 추출

[(2108, 0.4307822073084796),
 (772, 0.1660670340370092),
 (2143, 0.145095430011987),
 (364, 0.1405297418052926),
 (1385, 0.13624075366307076),
 (337, 0.11898561396522282),
 (1533, 0.1067728669872957),
 (2276, 0.10365129078035425),
 (2203, 0.09753591042961841),
 (1354, 0.08567801391582365)]

In [ ]:
# 인덱스를 Movie Title로 변환
sim_scores = [(movie2id[i], score) for i, score in sim_scores[0:30]]
sim_scores

[('Toy Story 2', 0.4307822073084796),
 ('Rebel Without a Cause', 0.1660670340370092),
 ('Man on the Moon', 0.145095430011987),
 ('Malice', 0.1405297418052926),
 ('Condorman', 0.13624075366307076),
 ('For Love or Money', 0.11898561396522282),
 ('Indecent Proposal', 0.1067728669872957),
 ('Bound for Glory', 0.10365129078035425),
 ("White Men Can't Jump", 0.09753591042961841),
 ("Child's Play 3", 0.08567801391582365),
 ('Radio Days', 0.0834858975130059),
 ('Window to Paris', 0.07808395765430066),
 ('Funny Farm', 0.07784485653742255),
 ('I Shot Andy Warhol', 0.07721830363477511),
 ('Mute Witness', 0.06374194834345086),
 ('Firestarter', 0.0636830549820565),
 ('Love! Valour! Compassion!', 0.06018735576749003),
 ('Dirty Work', 0.059132634013112514),
 ('Bringing Up Baby', 0.0566974071715919),
 ('Stepmom', 0.05540994987147116),
 ('Zachariah', 0.05378807220390342),
 ('Singles', 0.05169443233521159),
 ('Carried Away', 0.04910882416738239),
 ('Four Rooms', 0.04849504550720432),
 ('Bride of Re-Anim

In [ ]:
#Check EDA Notebook for how and why I got these indices.
md = md.replace([np.inf, -np.inf], np.nan)  # 무한대 값을 NaN으로 변환
md['id'] = md['id'].fillna(0).astype(int) # NaN값을 0으로 대체

md['id'] = md['id'].astype('int')

In [ ]:
smd = md

In [ ]:
t_data = smd

**영화 내용 기반 추천 시스템(Movie Description Based Recommender)**

In [ ]:
smd['tagline'] = smd['tagline'].fillna('')
smd['description'] = smd['overview'] + smd['tagline']
smd['description'] = smd['description'].fillna('')

smd['description'].head()

0    Led by Woody, Andy's toys live happily in his ...
1    When siblings Judy and Peter discover an encha...
2    A family wedding reignites the ancient feud be...
3    Cheated on, mistreated and stepped on, the wom...
4    Just when George Banks has recovered from his ...
Name: description, dtype: object

In [ ]:
# n-그램:단어장 생성에 사용할 토큰의 크기를 결정한다. 모노그램(1-그램)은 토큰 하나만 단어로 사용하며 바이그램(2-그램)은 두 개의 연결된 토큰을 하나의 단어로 사용한다.
# Stop Words:문서에서 단어장을 생성할 때 무시할 수 있는 단어를 말한다. 보통 영어의 관사나 접속사, 한국어의 조사 등이 여기에 해당한다. stop_words 인수로 조절할 수 있다.
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(smd['description'])

In [ ]:
print(tfidf_matrix[10])

In [ ]:
tfidf_matrix.shape

(3883, 92104)

**코사인 유사도(Cosine Similarity)**

In [ ]:
# linear_kernel는 두 벡터의 dot product 이다.
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [ ]:
cosine_sim[0]

array([1.        , 0.00823448, 0.        , ..., 0.        , 0.        ,
       0.        ])

In [ ]:
smd = smd.reset_index()
titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])

print(titles.head(), indices.head())

0                      Toy Story
1                        Jumanji
2               Grumpier Old Men
3              Waiting to Exhale
4    Father of the Bride Part II
Name: title, dtype: object title
Toy Story                      0
Jumanji                        1
Grumpier Old Men               2
Waiting to Exhale              3
Father of the Bride Part II    4
dtype: int64


In [ ]:
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [ ]:
get_recommendations('Toy Story').head(20)

3045              Toy Story 2
3105          Man on the Moon
1087    Rebel Without a Cause
1972                Condorman
449         For Love or Money
1224                Manhattan
486                    Malice
2363                  Stepmom
2200        Indecent Proposal
3194     White Men Can't Jump
1061                  Sleeper
594           Window to Paris
2681               Radio Days
756        I Shot Andy Warhol
1924           Child's Play 3
3302          Bound for Glory
593              Pretty Woman
2727               Funny Farm
1285             Paris, Texas
3639              Firestarter
Name: title, dtype: object

**메타 데이터 기반 추천시스템(Metadata Based Recommender)**

In [ ]:
smd = md

In [ ]:
smd

,MovieID,title,year,id,genres,popularity,overview,tagline,vote_average,vote_count,cast,crew,keywords,wr,description
0,1,Toy Story,1995,862,"[Animation, Comedy, Family]",21.946943,"Led by Woody, Andy's toys live happily in his ...",0,4.146846,2077.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,...",3.681488,"Led by Woody, Andy's toys live happily in his ..."
1,2,Jumanji,1995,8844,"[Adventure, Fantasy, Family]",17.015539,When siblings Judy and Peter discover an encha...,Roll the dice and unleash the excitement!,3.201141,701.0,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1...",2.937823,When siblings Judy and Peter discover an encha...
2,3,Grumpier Old Men,1995,15602,"[Romance, Comedy]",11.712900,A family wedding reignites the ancient feud be...,Still Yelling. Still Fighting. Still Ready for...,3.016736,478.0,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[{'id': 1495, 'name': 'fishing'}, {'id': 12392...",2.841801,A family wedding reignites the ancient feud be...
3,4,Waiting to Exhale,1995,31357,"[Comedy, Drama, Romance]",3.859495,"Cheated on, mistreated and stepped on, the wom...",Friends are the people who let you be yourself...,2.729412,170.0,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...","[{'id': 818, 'name': 'based on novel'}, {'id':...",2.757703,"Cheated on, mistreated and stepped on, the wom..."
4,5,Father of the Bride Part II,1995,11862,[Comedy],8.387519,Just when George Banks has recovered from his ...,Just When His World Is Back To Normal... He's ...,3.006757,296.0,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...","[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n...",2.815981,Just when George Banks has recovered from his ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3878,3948,Meet the Parents,2000,1597,"[Comedy, Romance]",14.178281,"Greg Focker is ready to marry his girlfriend, ...",First comes love. Then comes the interrogation.,3.635731,862.0,"[{'cast_id': 1, 'character': 'Gaylord ""Greg"" F...","[{'credit_id': '52fe4303c3a36847f8033ca9', 'de...","[{'id': 591, 'name': 'cia'}, {'id': 822, 'name...",3.155754,"Greg Focker is ready to marry his girlfriend, ..."
3879,3949,Requiem for a Dream,2000,641,"[Crime, Drama]",6.874828,The hopes and dreams of four ambitious people ...,0,4.115132,304.0,"[{'cast_id': 29, 'character': 'Sara Goldfarb',...","[{'credit_id': '52fe4263c3a36847f801a72b', 'de...","[{'id': 1803, 'name': 'drug addiction'}, {'id'...",3.065684,The hopes and dreams of four ambitious people ...
3880,3950,Tigerland,2000,10687,"[Drama, War]",5.878885,A group of recruits go through Advanced Infant...,The system wanted them to become soldiers. One...,3.666667,54.0,"[{'cast_id': 1, 'character': 'Pvt. Roland Bozz...","[{'credit_id': '52fe43a39251416c75018317', 'de...","[{'id': 10183, 'name': 'independent film'}, {'...",2.806454,A group of recruits go through Advanced Infant...
3881,3951,Two Family House,2000,63956,"[Drama, Romance]",0.265689,Buddy Visalo (Michael Rispoli) is a factory wo...,The only way to find out what you love is to r...,3.900000,40.0,"[{'cast_id': 1, 'character': 'Buddy Visalo', '...","[{'credit_id': '52fe46c2c3a368484e0a2471', 'de...",[],2.803971,Buddy Visalo (Michael Rispoli) is a factory wo...


**도메인 지식**

In [ ]:
import ast

smd['cast'] = smd['cast'].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else [])
smd['crew'] = smd['crew'].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else [])
smd['keywords'] = smd['keywords'].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else [])


In [ ]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [ ]:
smd['director'] = smd['crew'].apply(get_director)

In [ ]:
# 출연진 중 상위에 노출되는 3명만 추출
smd['cast'] = smd['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
smd['cast'] = smd['cast'].apply(lambda x: x[:3] if len(x) >= 3 else x)

In [ ]:
smd['keywords'] = smd['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [ ]:
# 출연진의 이름에서 공백 삭제
smd['cast'] = smd['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [ ]:
# 감독의 이름에서 공백 삭제 및 3번 언급?
smd['director'] = smd['director'].astype('str').apply(lambda x: str.lower(x.replace(" ", "")))
smd['director'] = smd['director'].apply(lambda x: [x, x, x])

**키워드(Keywords)**

In [ ]:
s = smd.apply(lambda x: pd.Series(x['keywords']), axis=1).stack().reset_index(level=1, drop=True)
s.name = 'keyword'

In [ ]:
s = s.value_counts()
s[:5]

independent film    228
woman director      171
murder              121
suspense             82
based on novel       80
Name: keyword, dtype: int64

In [ ]:
# 2번 이상 등장한 키워드만 추출
s = s[s > 1]

In [ ]:
# 어근 추출을 통해 동일 의미&다른 형태의 단어(dogs&dog, imaging&image 등)를 동일한 단어로 인식
stemmer = SnowballStemmer('english')
print("dogs의 어근 : ", stemmer.stem('dogs'))
print("dog의 어근 : ", stemmer.stem('dog'))

dogs의 어근 :  dog
dog의 어근 :  dog


In [ ]:
def filter_keywords(x):
    words = []
    for i in x:
        if i in s:
            words.append(i)
    return words

In [ ]:
# 키워드의 어근을 찾아서 공백 제거 후 세팅
smd['keywords'] = smd['keywords'].apply(filter_keywords)
smd['keywords'] = smd['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])
smd['keywords'] = smd['keywords'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [ ]:
# 키워드의 어근을 찾아서 공백 제거 후 세팅
smd['keywords'] = smd['keywords'].apply(filter_keywords)
smd['keywords'] = smd['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])
smd['keywords'] = smd['keywords'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [ ]:
smd['soup'] = smd['keywords'] + smd['cast'] + smd['director'] + smd['genres']
smd['soup'] = smd['soup'].apply(lambda x: ' '.join(x))

In [ ]:
count = CountVectorizer(analyzer='word', ngram_range=(1,2), min_df=0, stop_words='english')
count_matrix = count.fit_transform(smd['soup'])

In [ ]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [ ]:
smd = smd.reset_index()
titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])

In [ ]:
get_recommendations('Toy Story').head(10)

3045                 Toy Story 2
3360           Creature Comforts
1017               Pete's Dragon
1756            Meet the Deedles
156                       Casper
700             Oliver & Company
3682                 Chicken Run
2225                        Antz
1018    Bedknobs and Broomsticks
1459            Cats Don't Dance
Name: title, dtype: object

In [ ]:
get_recommendations('Jumanji').head(10)

1985              Honey, I Shrunk the Kids
2432                           October Sky
3420                                  Hook
584                                Aladdin
996               Escape to Witch Mountain
1974    Darby O'Gill and the Little People
2022            Return from Witch Mountain
626                All Dogs Go to Heaven 2
1018              Bedknobs and Broomsticks
1851                        Small Soldiers
Name: title, dtype: object

**인기도와 평점(Popularity and Ratings)**

In [ ]:
def improved_recommendations(title):
    print(title)
    idx = indices[title]
    print(idx)
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]
    print(movie_indices)

    movies = smd.iloc[movie_indices][['title','vote_count','vote_average','year']]
#     print(movies)

    vote_counts = movies[movies['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = movies[movies['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(0.60)
    qualified = movies[(movies['vote_count'] >= m) & (movies['vote_count'].notnull())]
#     print(qualified)
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['wr'] = qualified.apply(weighted_rating, axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(10)
    print(qualified)
    return qualified

In [ ]:
improved_recommendations('Toy Story')

Toy Story
0
[3045, 3360, 1017, 1756, 156, 700, 3682, 2225, 1018, 1459, 1029, 216, 1659, 2497, 3106, 3330, 667, 1490, 74, 245, 632, 856, 1099, 1967, 2838]
                         title  vote_count  vote_average  year        wr
3045               Toy Story 2        1585      4.218927  1999  3.637980
3106              Galaxy Quest        1728      3.771412  1999  3.389651
3682               Chicken Run        1329      3.879609  2000  3.386068
3360         Creature Comforts         274      4.335766  1990  3.087540
2225                      Antz         645      3.483721  1998  3.036566
1029        That Thing You Do!         528      3.481061  1996  3.002554
1018  Bedknobs and Broomsticks         319      3.479624  1971  2.929248
1017             Pete's Dragon         323      3.077399  1977  2.836329
156                     Casper         259      2.980695  1995  2.805444
667                  Space Jam         563      2.619893  1996  2.712626


,title,vote_count,vote_average,year,wr
3045,Toy Story 2,1585,4.218927,1999,3.637980
3106,Galaxy Quest,1728,3.771412,1999,3.389651
3682,Chicken Run,1329,3.879609,2000,3.386068
3360,Creature Comforts,274,4.335766,1990,3.087540
2225,Antz,645,3.483721,1998,3.036566
1029,That Thing You Do!,528,3.481061,1996,3.002554
1018,Bedknobs and Broomsticks,319,3.479624,1971,2.929248
1017,Pete's Dragon,323,3.077399,1977,2.836329
156,Casper,259,2.980695,1995,2.805444
667,Space Jam,563,2.619893,1996,2.712626


In [ ]:
improved_recommendations('Jumanji')

Jumanji
1
[1985, 2432, 3420, 584, 996, 1974, 2022, 626, 1018, 1851, 2018, 481, 2074, 3097, 1110, 1546, 2024, 1050, 1489, 542, 932, 1981, 2360, 2361, 1707]
                         title  vote_count  vote_average  year        wr
584                    Aladdin        1351      3.788305  1992  3.339244
2432               October Sky         588      4.137755  1999  3.255587
2018                 Peter Pan         594      3.659933  1953  3.086317
1546       Conan the Barbarian         572      3.421329  1982  2.994478
2074                    Legend         355      3.270423  1985  2.890533
3420                      Hook         722      3.018006  1991  2.866385
1985  Honey, I Shrunk the Kids        1045      2.933014  1989  2.847381
1851            Small Soldiers         364      2.810440  1998  2.774662
481           Last Action Hero         500      2.508000  1993  2.680332
542          Super Mario Bros.         350      1.874286  1993  2.540517


,title,vote_count,vote_average,year,wr
584,Aladdin,1351,3.788305,1992,3.339244
2432,October Sky,588,4.137755,1999,3.255587
2018,Peter Pan,594,3.659933,1953,3.086317
1546,Conan the Barbarian,572,3.421329,1982,2.994478
2074,Legend,355,3.270423,1985,2.890533
3420,Hook,722,3.018006,1991,2.866385
1985,"Honey, I Shrunk the Kids",1045,2.933014,1989,2.847381
1851,Small Soldiers,364,2.810440,1998,2.774662
481,Last Action Hero,500,2.508000,1993,2.680332
542,Super Mario Bros.,350,1.874286,1993,2.540517


## 평가지표 계산

In [ ]:
t_data

In [ ]:
b_movies = pd.read_csv('movies.csv', sep='::', names=['MovieID', 'Title', 'Genres'])

#따옴표 제거
b_movies['MovieID'] = b_movies['MovieID'].str.replace('"', '')
b_movies['Title'] = b_movies['Title'].str.replace('"', '')
b_movies['Genres'] = b_movies['Genres'].str.replace('"', '')

# 'MovieID' 열의 데이터 타입 변환
b_movies['MovieID'] = pd.to_numeric(b_movies['MovieID'], errors='coerce')
b_movies['MovieID'] = b_movies['MovieID'].astype(int)
b_movies

In [ ]:
b_movies['year'] = b_movies['Title'].str.extract(r'\((\d{4})\)')
b_movies['title'] = b_movies['Title'].str.replace(r"\(.*\)", "").str.strip()
b_movies = b_movies[['MovieID', 'title', 'year', 'Genres']]

In [ ]:
b_movies

In [ ]:
b_ratings = pd.read_csv('ratings.csv', sep='::', names=['UserID', 'MovieID', 'Rating', 'TimeStamp'])
b_ratings

In [ ]:
b_users = pd.read_csv('users.csv', sep='::', names=['UserID', 'Gender', 'Age', 'Occupation', 'Zip-code'])
b_users

In [ ]:
b_rat_user = pd.merge(b_ratings, b_users, on='UserID', how='left')
b_total = pd.merge(b_rat_user, b_movies, on='MovieID', how='left')
b_total = b_total[['UserID', 'MovieID', 'title', 'year', 'Genres', 'Rating', 'TimeStamp', 'Gender', 'Age', 'Occupation', 'Zip-code']]

In [ ]:
b_total

In [ ]:
b_total['TimeStamp'] = pd.to_datetime(b_total['TimeStamp'], unit='s')
b_total['Date'] = b_total['TimeStamp'].dt.date
b_total['Time'] = b_total['TimeStamp'].dt.time

In [ ]:
b_total = b_total[['UserID', 'MovieID', 'title', 'year', 'Genres', 'Rating', 'Date', 'Time', 'Gender', 'Age', 'Occupation', 'Zip-code']]
b_total

In [ ]:
b_total.info()

In [ ]:
User1 = b_total[b_total['UserID'] == 1]
User1 = User1.sort_values(by=['Rating', 'Date', 'Time'], ascending=[False, False, False])
User1

In [ ]:
User1.shape

In [ ]:
improved_recommendations('Pocahontas')

실제 본 영화와 추천영화 비교? 하는 ndcg & recall 계산